<a href="https://colab.research.google.com/github/Zesmat/Market-Price-Prediction/blob/main/Market_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model aims to predict market price based on given data set using Long Short Term Memory Algorithm

## **Data Preprocessing**

### **Load dataset**

In [37]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM


data = pd.read_csv(r'/content/drive/MyDrive/Market Price Prediction/MarketPricePrediction.csv')
#creating dataframe
df=data.sort_index(ascending=True, axis=0)
new_df=pd.DataFrame(index=range(0,len(data)),columns=['date', 'priceMax'])
for i in range(0, len(df)):
  new_df['date'][i]=df['date'][i]
  new_df['priceMax'][i]=df['priceMax'][i]
data.head()

,market,month,year,quantity,priceMin,priceMax,priceMod,state,city,date
0,ABOHAR(PB),January,2005,2350,404,493,446,PB,ABOHAR,January-2005
1,ABOHAR(PB),January,2006,900,487,638,563,PB,ABOHAR,January-2006
2,ABOHAR(PB),January,2010,790,1283,1592,1460,PB,ABOHAR,January-2010
3,ABOHAR(PB),January,2011,245,3067,3750,3433,PB,ABOHAR,January-2011
4,ABOHAR(PB),January,2012,1035,523,686,605,PB,ABOHAR,January-2012


### **Label Encoding**

In [16]:
from sklearn.preprocessing import LabelEncoder

label_encoder=LabelEncoder()

data['City_Encoded']=label_encoder.fit_transform(data['city'].astype(str))

### **Handling Missing Values**

*  Identify missing values in the dataset.

*  Decide whether to impute missing values or remove rows/columns with missing values based on the amount of missing data and its impact on the analysis.


In [38]:
import matplotlib.pyplot as plt

# Assuming you already have the 'data' DataFrame with the 'city_encoded' column
new_df.index=new_df.date
new_df.drop('date', axis=1, inplace=True)


data['date'] = pd.to_datetime(data['date'], format='%B-%Y')
data.index = data['date']




### **Feature Scaling**

In [34]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler(data['City_Encoded'])


## **Model Building**

In [ ]:
#creating train and test sets
dataset = new_df.values